# CSC 480-F25 Lab 10: AI and Humor

### Authors:
***[Team Member 1], [Team Member 2], [Team Member 3]***

California Polytechnic State University, San Luis Obispo;
Computer Science & Software Engineering Department

### Overview
This lab explores **AI and Humor** through a multi-agent system where a stand-up comic agent creates jokes and an audience agent provides feedback. We use the **agentic design approach** with AutoGen to iteratively refine jokes through agent-to-agent collaboration until the audience is satisfied.

---

## Environment Setup and Imports

In [ ]:
# Install required packages
%pip install "autogen-core" "autogen-agentchat" "autogen-ext[openai,azure]"

In [ ]:
import os
import asyncio
from typing import List

# Import AutoGen classes
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

### Azure OpenAI Configuration

In [ ]:
# Configure your Azure OpenAI client
azure_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")

# Ensure your API key is set as an environment variable
api_key = os.getenv("AZURE_SUBSCRIPTION_KEY")

client = AzureOpenAIChatCompletionClient(
    azure_deployment=azure_deployment,
    model="gpt-5-mini",
    api_version=api_version,
    azure_endpoint=azure_endpoint,
    api_key=api_key,
)

---

## Part 1: Agent Setup and Role Definition

We'll create two agents:
1. **Stand-Up Comic Agent**: Creates and refines jokes based on audience feedback
2. **Audience Agent**: Evaluates jokes and provides constructive feedback, ultimately deciding when a joke is funny enough

The agents will collaborate in a round-robin fashion, iteratively improving the joke until the audience is satisfied.

In [ ]:
# Define the Stand-Up Comic Agent
comic_agent = AssistantAgent(
    name="StandUp_Comic",
    system_message="""You are a creative stand-up comedian. Your job is to create funny, original jokes.
    
    When you first receive a topic, create an initial joke about it. When you receive feedback from the audience,
    carefully consider their critique and refine your joke to make it funnier. You can adjust the setup, punchline,
    timing, wordplay, or overall structure based on the feedback.
    
    Always present your joke clearly, and be open to iterating multiple times until you nail it.
    End your response with just the joke (no meta-commentary) when you're presenting it.""",
    model_client=client,
)

# Define the Audience Agent
audience_agent = AssistantAgent(
    name="Audience_Member",
    system_message="""You are a discerning comedy audience member who provides constructive feedback on jokes.
    
    For each joke you hear, evaluate it based on:
    - Setup and punchline structure
    - Timing and delivery
    - Originality and creativity
    - Overall humor and impact
    
    Provide specific, actionable feedback to help the comedian improve. Point out what works and what doesn't.
    Be honest but constructive.
    
    When a joke is genuinely funny and well-crafted, you should signal your approval by ending your response with:
    "APPROVED: This joke is funny enough!"
    
    Don't approve a joke the first time around - be tough on the comedian at first. But also don't be impossible to please.""",
    model_client=client,
)

---

## Part 2: Round-Robin Joke Refinement Pipeline

Now we'll set up the round-robin collaboration where:
1. The comic creates an initial joke
2. The audience provides feedback
3. The comic refines the joke based on feedback
4. Steps 2-3 repeat until the audience approves

We use a `TextMentionTermination` condition to stop when the audience says "APPROVED".

In [ ]:
async def run_comedy_workshop(topic: str, max_turns: int = 10):
    """
    Run a comedy workshop where a stand-up comic and audience collaborate to create a great joke.
    
    Args:
        topic: The topic for the joke
        max_turns: Maximum number of turns before giving up
    """
    
    # Define termination condition - stop when audience approves
    termination_condition = TextMentionTermination("APPROVED")
    
    # Create the round-robin group chat
    group_chat = RoundRobinGroupChat(
        participants=[comic_agent, audience_agent],
        termination_condition=termination_condition,
        max_turns=max_turns,
    )
    
    # Initial task for the comic
    task = f"Create a funny joke about: {topic}"
    
    print(f"Starting Comedy Workshop")
    print(f"Topic: {topic}")
    print(f"Max turns: {max_turns}")
    print("=" * 70)
    
    # Run the collaboration
    result = await group_chat.run(task=task)
    
    print("\n" + "=" * 70)
    print(f"Workshop completed after {len(result.messages)} messages")
    
    # Display the conversation
    print("\n Full Conversation:")
    print("=" * 70)
    for i, message in enumerate(result.messages, 1):
        print(f"\n[Message {i}] {message.source}:")
        print(message.content)
        print("-" * 70)
    
    return result

---

## Part 3: Running the Comedy Workshop

Let's test the system with different joke topics to see how the agents collaborate.

In [ ]:
# Example 1: AI and Programming
result1 = await run_comedy_workshop("artificial intelligence and programming", max_turns=10)

In [ ]:
# Example 2: College Life
result2 = await run_comedy_workshop("college students and finals week", max_turns=10)

In [ ]:
# Example 3: Your own topic - customize this!
result3 = await run_comedy_workshop("your topic here", max_turns=10)

---

## Part 4: Reflection & Analysis

### Agent Collaboration Effectiveness

**What worked well in the multi-agent comedy system?**

_(Reflect on how the agents communicated, whether the feedback was constructive, and if the iterative refinement process improved the jokes. Consider the clarity of roles and the effectiveness of the termination condition.)_

---

**Quality of Humor Generation**

_(Evaluate the final jokes produced by the system. Were they genuinely funny? How did they compare to the initial attempts? Did the refinement process actually improve the jokes, or did it sometimes make them worse?)_

---

**Potential Improvements**

_(Suggest ways to enhance the system. Could additional agents help? Would different termination conditions work better? How might you incorporate more diverse feedback or test jokes with different audience personas?)_

---

## Optional Bonus: Experimenting with Different Configurations

Try modifying the system to explore different configurations:
- Add a third agent (e.g., a "Comedy Coach" who provides structural advice)
- Adjust the audience's strictness by modifying their system message
- Change the termination condition (e.g., require multiple rounds of approval)
- Test with different types of humor (puns, observational, dark comedy, etc.)

In [ ]:
# Experiment space - try your own configurations here!